In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import seaborn as sns
import os
import time
import pyodbc
import warnings
from datetime import datetime, timedelta
import gc
import glob
import hashlib
from sklearn.feature_extraction.text import TfidfVectorizer
from unidecode import unidecode
import re

warnings.filterwarnings('ignore', message='pandas only supports SQLAlchemy connectable .*')
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', None)
# suppress scientific notation by setting float_format
pd.options.display.float_format = '{:.0f}'.format

## Jobs

In [2]:
# Read Excel file
df = pd.read_excel('datasets/jirka/czechitas_dw_public_IT_jobs_scraping.xlsx', sheet_name='Result 1')

In [3]:
df = df[df["source"]=="jobs_cz"]
# Convert the 'date_start' column to datetime format
df['date_start'] = pd.to_datetime(df['date_start'])
# Filter rows where 'date_start' is greater or equal to '2023-01-01'
df = df[df['date_start'] >= '2022-01-01']

In [4]:
# Hash the 'event_id' column
df['job_id'] = df['id'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())

In [5]:
df.head(3)

,source,id,name,company,city,county,CZK_salary_from,CZK_salary_to,url,date_start,date_end,junior,maternity,remote_partly,remote_fully,part_time,full_time,edu_basic,edu_high,edu_university,category,Kategorie_by_Czechitas,date_difference,date_difference_range,date_posted,date_posted_range,job_id
0,jobs_cz,1590329021,Security Specialist,Manuvia,Praha,Hlavní město Praha,NaN,NaN,https://www.jobs.cz/rpd/1590329021/,2022-05-10,2022-09-06,False,False,False,False,False,True,False,False,False,IT Security Specialist,IT security,119,61+,517,31+,2ff29b0b054abcd27e49693c1df601d22815cfa310374a...
1,jobs_cz,1591719727,C# .NET Developer pro IB Commodities Reporting,"COMMERZBANK Aktiengesellschaft, pobočka Praha",Praha,Hlavní město Praha,NaN,NaN,https://www.jobs.cz/rpd/1591719727/,2022-05-26,2022-08-25,False,False,True,False,False,True,False,False,False,C# vývojář,Back-end Software development,91,61+,501,31+,1777f9582bdd7f52ff1e0e967c3c783973d5e46d11859b...
2,jobs_cz,1598676535,Business / IT Analyst,Cleverlance Enterprise Solutions s.r.o.,Praha,Hlavní město Praha,65000,110000,https://www.jobs.cz/rpd/1598676535/,2022-08-24,2022-09-22,False,False,False,False,False,True,False,True,False,Business analytik,Business analyst (consultant),29,15-30,411,31+,12213571c5ebc7eccd8f67684566a1e138a9c0fe219255...


In [6]:
df['name'] = df['name'].str.lower()

In [7]:
# jobs = df[["id", "name"]].drop_duplicates().groupby("name").size().reset_index().sort_values(0, ascending=False)

In [8]:
# Define a function for custom job categorization based on the conditions provided
def custom_job_categorization(job_name_lower):
        
    if "test" in job_name_lower or "quality" in job_name_lower or "qa" in job_name_lower:
        return "Testing"
    
    elif (("javascript" in job_name_lower) or
          ("php" in job_name_lower) or
          ("front" in job_name_lower and "end" in job_name_lower) or
          (".js" in job_name_lower) or
          ("wordpress" in job_name_lower) or
          ("angular" in job_name_lower) or
          ("react" in job_name_lower) or
          ("web" in job_name_lower and ("develop" in job_name_lower or "program" in job_name_lower or "master" in job_name_lower or "vývoj" in job_name_lower or "kod" in job_name_lower))):
        return "Web & Frontend Development"
    
    elif ("c#" in job_name_lower or
          ".net" in job_name_lower or
          "java" in job_name_lower or
          "javist" in job_name_lower or
          "javě" in job_name_lower or
          "python" in job_name_lower or
          "c++" in job_name_lower or
          ("back" in job_name_lower and "end" in job_name_lower) or
          "software" in job_name_lower):
        return "Backend Development"
    
    elif (("machine" in job_name_lower and "learning" in job_name_lower)  or
          ("datov" in job_name_lower and "věd" in job_name_lower)  or
          "ai/ml" in job_name_lower or
          "ai " in job_name_lower or
          "data science" in job_name_lower or
          "artificial" in job_name_lower or
          ("stroj" in job_name_lower and "učen" in job_name_lower)):
        return "Data Science / Machine Learning / Artificial Intelligence"
    
    elif (("business" in job_name_lower and ("anal" in job_name_lower or "intell" in job_name_lower)) or
          "reporting" in job_name_lower or
          "sql" in job_name_lower or
          "qlik" in job_name_lower or
          "big data" in job_name_lower):
        return "Data Analytics / Business Intelligence / Reporting"
    
    elif ("recruit" in job_name_lower or
      ("back" in job_name_lower and "office" in job_name_lower) or
      "konstrukt" in job_name_lower or
      "r&d" in job_name_lower or
      "compliance" in job_name_lower):
        return "Not Relevant Positions"
    
    elif "security" in job_name_lower or "kyber" in job_name_lower or "cyber" in job_name_lower or "bezpeč" in job_name_lower or "data privacy" in job_name_lower or "data protection" in job_name_lower:
        return "Cybersecurity"
    
    elif "android" in job_name_lower or "ios" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "salesforce" in job_name_lower:
        return "Marketing Technology"
    
    elif "sap" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "proje" in job_name_lower or "produ" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "scrum" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "marke" in job_name_lower or "social" in job_name_lower or "ppc" in job_name_lower or "seo" in job_name_lower or "rtb" in job_name_lower:
        return "Marketing Technology"
    
    elif "dwh" in job_name_lower or "datab" in job_name_lower or "db " in job_name_lower:
        return "Data Analytics / Business Intelligence / Reporting"

    elif (("it" in job_name_lower and "spec" in job_name_lower) or
          "it analy" in job_name_lower or
          ("it" in job_name_lower and "support" in job_name_lower) or
          ("it" in job_name_lower and "admin" in job_name_lower) or
          ("it" in job_name_lower and "správc" in job_name_lower) or
          ("it" in job_name_lower and "techni" in job_name_lower) or
          "devops" in job_name_lower or
          ("it" in job_name_lower and "syst" in job_name_lower) or
          "it " in job_name_lower):
        return "Not Relevant Positions"
    
    elif ("anal" in job_name_lower or
          "data " in job_name_lower or
          "datový " in job_name_lower or
          "datová " in job_name_lower or
          (("bi" in job_name_lower or "dat" in job_name_lower or "dát" in job_name_lower) and
           ("power" in job_name_lower or
            "onsult" in job_name_lower or
            "anal" in job_name_lower or
            "onzult" in job_name_lower or
            "develop" in job_name_lower or
            "data" in job_name_lower or
            "exp" in job_name_lower or
            "design" in job_name_lower or
            "program" in job_name_lower or
            "manag" in job_name_lower or
            "admin" in job_name_lower or
            "archit" in job_name_lower or
            "engineer" in job_name_lower or
            "inž" in job_name_lower or
            "mod" in job_name_lower or
            "special" in job_name_lower or
            "lead" in job_name_lower or
            "etl" in job_name_lower))):
        return "Data Analytics / Business Intelligence / Reporting"
    
    elif ("graph" in job_name_lower or
          "graf" in job_name_lower or
          " ux" in job_name_lower or
          "ux " in job_name_lower or
          "design" in job_name_lower or
          " ui" in job_name_lower or
          "ui " in job_name_lower or
          "user experience" in job_name_lower):
        return "UX / UI / Graphic Design"
    
    elif ("full" in job_name_lower and "stack" in job_name_lower) or "fullstack" in job_name_lower:
        return "Backend Development"
    else:
        return "Not Relevant Positions"

In [9]:
df['job_category'] = df['name'].apply(custom_job_categorization)

In [10]:
df = df[["job_id", "date_start", "job_category"]]

In [11]:
df.to_csv("job_advertisements.csv", index=False)

## Courses

In [12]:
# Read Excel file
df_event = pd.read_excel('datasets/jirka/v_event.xlsx', sheet_name='Result 1')

In [13]:
# Convert the 'Nazev_Produktu' column to lowercase
df_event['nazev_produktu'] = df_event['nazev_produktu'].str.lower()
df_event['cycle'] = df_event['cycle'].str.lower()

In [14]:
df_event = df_event[df_event["cycle"]!="ostatní"]
df_event = df_event[df_event["cycle"]!="komunitní akce"]

In [15]:
def update_cycle_names(cycle_name):
    # Create a mapping dictionary for the name changes
    name_changes = {
        'tvorba webu': 'tvořím web',
        'testování': 'testuju',
        'programování': 'programuju',
        # Add more changes here as needed
    }
    
    # Check if the cycle_name exists in the dictionary keys and update accordingly
    return name_changes.get(cycle_name, cycle_name)

# Apply the function to update the 'cycle' names
df_event['cycle'] = df_event['cycle'].apply(update_cycle_names)

In [16]:
# Convert the 'start_at' and 'end_at' columns to datetime type if they are not already
df_event['start_at'] = pd.to_datetime(df_event['start_at'])
df_event['end_at'] = pd.to_datetime(df_event['end_at'])

# Filter the rows where 'start_at' is greater than '2022-01-01'
df_event = df_event[df_event['start_at'] >= '2020-01-01']

# filter hotova
df_event = df_event[~df_event["event_state"].str.startswith(("zrušená", "zamítnutá"))]
df_event = df_event[df_event["teaching_hours"]>0]
df_event = df_event[~df_event['nazev_produktu'].str.contains("učitel", case=False)]
df_event = df_event[~df_event['nazev_produktu'].str.contains("meetup", case=False)]
df_event = df_event[~df_event['nazev_produktu'].str.contains("meet up", case=False)]
df_event = df_event[~df_event['nazev_produktu'].str.contains("job fair", case=False)]

In [17]:
# df_event["event_state"].drop_duplicates()

In [18]:
# Define a function to determine the course type
def assign_state_corr(event_state):
    if event_state == "hotová" or event_state == "vyhodnocení":
        return "completed"
    elif event_state == "probíhá":
        return "ongoing"
    else:
        return "upcoming"

# Apply the function to create the new 'course_type' column
df_event['event_state_english'] = df_event["event_state"].apply(assign_state_corr)

In [19]:
# # Define a function to determine the course type
# def get_course_type(row):
#     if row['nazev_produktu'].startswith("digitální akademie"):
#         return "Digital Academy"
#     elif row['start_at'] == row['end_at']:
#         return "One Day Course"
#     else:
#         return "Multiple Days Course"

# # Apply the function to create the new 'course_type' column
# df_event['course_type'] = df_event.apply(get_course_type, axis=1)

In [20]:
# # Define a function to determine the course level based on 'nazev_produktu'
# def get_course_level(nazev_produktu):
#     if ("2" in nazev_produktu or
#           ("3" in nazev_produktu and "365" not in nazev_produktu) or
#           "digitální akademie" in nazev_produktu or
#           "data science" in nazev_produktu or
#           "objektově orientované" in nazev_produktu or
#           "vba" in nazev_produktu):
#         return "Advanced"
    
#     elif ("úvod" in nazev_produktu or
#         "1" in nazev_produktu or
#         "poznej" in nazev_produktu or
#         "základ" in nazev_produktu or
#         "start" in nazev_produktu or
#         "staň" in nazev_produktu or
#          "agilní vývoj: scrum" in nazev_produktu):
#         return "Beginner"
   
#     else:
#         return "All Levels"

# # Apply the function to create a new 'course_level' column
# df_event['course_level'] = df_event['nazev_produktu'].apply(get_course_level)

In [21]:
# Define a function to assign categories based on 'cycle' and 'nazev_produktu'
def assign_job_matching_category(row):
    if ("seo" in row['nazev_produktu'] or
        "copywriting" in row['nazev_produktu']):
        return "Marketing Technology"
    
    elif 'javascript' in row['nazev_produktu'] or 'wordpress' in row['nazev_produktu']:
        return "Web & Frontend Development"
    
    elif ('java' in row['nazev_produktu'] or
        'c#' in row['nazev_produktu']):
        return 'Backend Development'
    
    elif ('bezpeč' in row['nazev_produktu']):
        return 'Cybersecurity'
    
    elif ('data science' in row['nazev_produktu']):
        return 'Data Science / Machine Learning /Artificial Intelligence'
    
    elif ("figma" in row['nazev_produktu'] or
          "canva" in row['nazev_produktu'] or
          "ux" in row['nazev_produktu']):
        return "UX / UI / Graphic Design"
    
    elif (" git" in row['nazev_produktu'] or "it analýza" in row['nazev_produktu'] or "devops" in row['nazev_produktu']):
        return "Other"
    
    elif ("scrum" in row['nazev_produktu']):
        return "Other"
    
    elif ("salesforce" in row['nazev_produktu']):
        return "Marketing Technology"
    
    elif ("tvořím web" in row['cycle']):
        return "Web & Frontend Development"
    
    elif ("programuju" in row['cycle']):
        return "Backend Development"
    
    elif ("testuju" in row['cycle']):
        return "Testing"
    
    elif ("kybernetická bezpečnost" in row['cycle']):
        return "Cybersecurity"
    
    elif 'datová analýza' in row['cycle']:
        return 'Data Analytics / Business Intelligence / Reporting'
    
    elif 'kariér' in row['cycle']:
        return 'Career'
    
    else:
        return 'Other'

# Apply the function to create a new column in the DataFrame
df_event['job_category'] = df_event.apply(assign_job_matching_category, axis=1)

In [22]:
df_event = df_event[df_event["job_category"]!="Other"]
df_event = df_event[df_event["job_category"]!="Career"]
# Filter out rows where 'nazev_produktu' contains "kroužek"
df_event = df_event[~df_event['nazev_produktu'].str.contains('kroužek')]

In [23]:
relevant_events = df_event[["event_id", "job_category"]].drop_duplicates().reset_index()

In [24]:
# Hash the 'event_id' column
df_event['course_id'] = df_event['event_id'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())

In [25]:
df_event = df_event[["course_id", "nazev_produktu", "start_at", "end_at", "event_state_english", "teaching_hours", "job_category", "study_form"]]

In [26]:
df_event.to_csv("courses.csv", index=False)

## Participants

In [27]:
# Read Excel file
df_participant = pd.read_excel('datasets/jirka/participant_registration.xlsx', sheet_name='Result 1')

In [28]:
df_participant.head(3)

,user_id,event_participant_id,event_id,age_group,region,technology_experience,motivation,expectation,participant_state,alumni_state
0,28274,180919,1938,19-26,Hlavní město Praha,NaN,NaN,"Je to směr, kterým se chci vydat? Bavilo by mě...",Zúčastněný,True
1,19503,176412,1790,27-35,Hlavní město Praha,NaN,NaN,NaN,Nezúčastněný,False
2,28273,180917,1907,NaN,Hlavní město Praha,zkousela_jsme,chci_lepsi_uplatneni,"Větší přehled o datových modelech, základy SQL...",Zúčastněný,True


In [29]:
df_uplatnene = pd.read_excel('datasets/jirka/car_uplatnene.xlsx', sheet_name='Result 1')

In [30]:
# Add a new column "IT_job" with all values set to True
df_uplatnene['got_IT_job_after_digital_academy'] = True

In [31]:
df_event_participants = relevant_events.merge(df_participant, on="event_id", how="left")

In [32]:
df_event_part_job = df_event_participants.merge(df_uplatnene, on="user_id", how="left")

In [33]:
# Fill null values in the "IT_job" column with False
df_event_part_job['got_IT_job_after_digital_academy'].fillna(False, inplace=True)

In [34]:
df_event_part_job = df_event_part_job[df_event_part_job['participant_state'].notna()]

In [35]:
# Define a function to determine the course type
def assign_participant_state(participant_state):
    if "Zrušený" in participant_state:
        return "Registration Cancelled"
    elif participant_state == "Zúčastněný":
        return "Participated"
    elif participant_state == "Nezúčastněný":
        return "Did Not Participate"
    elif "Nepřijatý" in participant_state:
        return "Not Accepted"
    elif participant_state == "Přijatý":
        return "Accepted"
    elif participant_state == "Přihlášený":
        return "Applied"
    else:
        return "Other"

# Apply the function to create the new 'course_type' column
df_event_part_job['participant_state_english'] = df_event_part_job["participant_state"].apply(assign_participant_state)

In [36]:
df_event_part_job.head(3)

,index,event_id,job_category,user_id,event_participant_id,age_group,region,technology_experience,motivation,expectation,participant_state,alumni_state,got_IT_job_after_digital_academy,participant_state_english
0,75,77,Backend Development,2204,2518,NaN,NaN,nevim_o_co_jde,technologie_se_mi_hodi,"Úvod do programování, zjistit co a kde si dále...",Zúčastněný,True,False,Participated
1,75,77,Backend Development,2291,2650,NaN,NaN,zkousela_jsme,chci_lepsi_uplatneni,"Že se dozvím dové věci a nakopne mě k tomu, ab...",Zúčastněný,True,False,Participated
2,75,77,Backend Development,1565,3706,NaN,NaN,zkousela_jsme,chci_praci_v_tomto_oboru,"Naučení základů C#, na který budu po kurzu moc...",Zrušený účastníkem,False,False,Registration Cancelled


In [37]:
# Hash the 'event_id' column
df_event_part_job['course_id'] = df_event_part_job['event_id'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
df_event_part_job['participant_id'] = df_event_part_job['event_participant_id'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())

In [38]:
relevant_participants = df_event_part_job[["course_id", "participant_id", "age_group", "motivation", "participant_state_english", "got_IT_job_after_digital_academy"]]

In [39]:
relevant_participants.to_csv("participants.csv", index=False)

In [40]:
relevant_motivation = df_event_part_job[df_event_part_job["job_category"]=="Data Analytics / Business Intelligence / Reporting"][["participant_id", "expectation"]]

In [41]:
relevant_motivation = relevant_motivation[relevant_motivation["expectation"].notna()]

In [42]:
relevant_participants.head(3)

,course_id,participant_id,age_group,motivation,participant_state_english,got_IT_job_after_digital_academy
0,a88a7902cb4ef697ba0b6759c50e8c10297ff58f942243...,1ca4cb56d951df8df2bc741ecf0b3441728eba835c6711...,NaN,technologie_se_mi_hodi,Participated,False
1,a88a7902cb4ef697ba0b6759c50e8c10297ff58f942243...,19bceca6aae94d6c94628fc3c24b7a8f206dae77e2f058...,NaN,chci_lepsi_uplatneni,Participated,False
2,a88a7902cb4ef697ba0b6759c50e8c10297ff58f942243...,995d8c2b85ecc1775188618e736639d6289693882e240e...,NaN,chci_praci_v_tomto_oboru,Registration Cancelled,False


In [43]:
len(relevant_motivation)

17384

In [44]:
# participant_motivation = df_event_part_job[["course_id", "participant_id", "expectation"]]
# participant_motivation.to_csv("word_motiv.csv", index=False)
# participant_motivation.head(100).to_csv("word_motiv_sample.csv", index=False)

### Participants motivation

In [45]:
df_motiv_czech_filtered = relevant_motivation

# Replace null values in 'expectation' with a unique placeholder
# placeholder = "NoExpectationProvided"
# df_motiv_czech_filtered['expectation'].fillna(placeholder, inplace=True)

# Custom list of Czech stop words provided by the user
custom_stop_words = ["06",
                     "10",
                     "11",
                     "2a3",
                     "2d",
                     "31",
                     "365",
                     "5let",
                     "39",
                     "40",
                     "3jku",
                     "3mesicnim",
                     "3měsíčním",
                     "409007",
                     "41",
                     "42",
                     "43",
                     "44",
                     "45",
                     "46",
                     "47",
                     "48",
                     "50",
                     "50tce",
                     "55",
                     "5ti",
                     "3x",
                     "5x",
                     "6e",
                     "77",
                     "80",
                     "12",
                     "12ti",
                     "13",
                     "13ti",
                     "14",
                     "14ti",
                     "15",
                     "15ti",
                     "16",
                     "16ti",
                     "17",
                     "17ti",
                     "18",
                     "18ti",
                     "19",
                     "19ti",
                     "20",
                     "21",
                     "24",
                     "25",
                     "26",
                     "20ti",
                     "100",
                     "150",
                     "600",
                     "900",
                     "1000",
                     "1995",
                     "2020",
                     "2021",
                     "2022",
                     "2023",
                     "2024",
                     "10000",
                     "15000",
                     "10denniho",
                     "10denního",
                     "aby",
                     "abz",
                     "abych",
                     "abch",
                     "ac",
                     "ač",
                     "alespoň",
                     "alespon",
                     "aspoň",
                     "aspon",
                     "analýza",
                     "analyza",
                     "analýze",
                     "analyze",
                     "analyzu",
                     "analýzu",
                     "analýzy",
                     "analyzy",
                     "analyzou",
                     "analýzou",
                     "analytice",
                     "analytika",
                     "analyzovat",
                     "analytik",
                     "analyticky",
                     "analytický",
                     "analytičky",
                     "analytiky",
                     "analýz",
                     "analyz",
                     "analyzach",
                     "analýzách",
                     "akademii",
                     "azure",
                     "to",
                     "bi",
                     "by",
                     "bych",
                     "byt",
                     "být",
                     "bylo",
                     "bude",
                     "cemu",
                     "čemu",
                     "čištění",
                     "cisteni",
                     "cele",
                     "celé",
                     "cokoliv",
                     "cokoli",
                     "co",
                     "čo",
                     "časem",
                     "casem",
                     "cem",
                     "čem",
                     "da",
                     "dá",
                     "dal",
                     "dál",
                     "dále",
                     "dale",
                     "dalsi",
                     "další",
                     "dalsich",
                     "dalších",
                     "dalsiho",
                     "dalšího",
                     "dane",
                     "dané",
                     "daném",
                     "danem",
                     "databázi",
                     "dátabázi",
                     "dátábází",
                     "databází",
                     "databazi",
                     "databazemi",
                     "databázemi",
                     "databáze",
                     "databaze",
                     "databazich",
                     "databázích",
                     "databázím",
                     "databazim",
                     "datovou",
                     "dat",
                     "datům",
                     "datum",
                     "daty",
                     "data",
                     "datova",
                     "datová",
                     "datovy",
                     "datový",
                     "datoveho",
                     "datového",
                     "datové",
                     "datove",
                     "datech",
                     "dělat",
                     "delat",
                     "db",
                     "digitální",
                     "digitalni",
                     "díky",
                     "diky",
                     "datum",
                     "datovými",
                     "datovymi",
                     "datovej",
                     "danou",
                     "danému",
                     "danemu",
                     "dělá",
                     "dela",
                     "ho",
                     "excel",
                     "excell",
                     "excelem",
                     "excelu",
                     "excellu",
                     "funkci",
                     "funkcí",
                     "funkcích",
                     "funkcich",
                     "fce",
                     "grafy",
                     "hlavne",
                     "hlavně",
                     "chci",
                     "chtěla",
                     "chtela",
                     "chcela",
                     "že",
                     "som",
                     "své",
                     "jak",
                     "000",
                     "sa",
                     "se",
                     "jsem",
                     "čem",
                     "ze",
                     "že",
                     "zda",
                     "nich",
                     "budu",
                     "ako",
                     "idealne",
                     "ideálně",
                     "jazyce",
                     "jazyk",
                     "jazyku",
                     "jeho",
                     "ještě",
                     "jeste",
                     "jejim",
                     "jejím",
                     "je",
                     "jestli",
                     "jít",
                     "jit",
                     "jiz",
                     "již",
                     "když",
                     "kdyz",
                     "konkretni",
                     "konkrétní",
                     "konkrétně",
                     "konkretne",
                     "kontingenčními",
                     "kontingencnimi",
                     "kontingenční",
                     "kontingencni",
                     "kurzem",
                     "li",
                     "lze",
                     "mit",
                     "mít",
                     "moc",
                     "mnohem",
                     "muze",
                     "může",
                     "mohlo",
                     "nejvice",
                     "nejvíce",
                     "něm",
                     "nem",
                     "nemám",
                     "nemam",
                     "ním",
                     "nim",
                     "6",
                     "viz",
                     "jaké",
                     "kontingencni",
                     "kontingenční",
                     "pc",
                     "právě",
                     "prave",
                     "při",
                     "pri",
                     "makra",
                     "me",
                     "mě",
                     "ms",
                     "mš",
                     "mne",
                     "mně",
                     "moct",
                     "můžu",
                     "muzu",
                     "mohu",
                     "modely",
                     "ne",
                     "nástroje",
                     "nastroje",
                     "nástrojem",
                     "nastrojem",
                     "nástroji",
                     "nastroji",
                     "nej",
                     "něj",
                     "něco",
                     "neco",
                     "nejake",
                     "nějaké",
                     "nejvic",
                     "nejvíc",
                     "nejlépe",
                     "nejlepe",
                     "nadále",
                     "nadale",
                     "vse",
                     "vše",
                     "opravdu",
                     "ohledne",
                     "ohledně",
                     "oop",
                     "popřípadě",
                     "popripade",
                     "poradne",
                     "prikaz",
                     "příkaz",
                     "presne",
                     "přesně",
                     "pro",
                     "programovacího",
                     "programovaciho",
                     "programovací",
                     "programovaci",
                     "programování",
                     "programovani",
                     "programováni",
                     "programovat",
                     "programu",
                     "programem",
                     "poradne",
                     "pořádně",
                     "pujde",
                     "půjde",
                     "phyton",
                     "phytonu",
                     "python",
                     "pythonu",
                     "pythonem",
                     "power",
                     "powerbi",
                     "pbi",
                     "primárně",
                     "primarne",
                     "případně",
                     "pripadne",
                     "případné",
                     "příkaz",
                     "prikaz",
                     "príkaz",
                     "příkazu",
                     "příkazů",
                     "přikážu",
                     "především",
                     "predevsim",
                     "co",
                     "da",
                     "ci",
                     "či",
                     "jejich",
                     "jejích",
                     "jsou",
                     "ji",
                     "jí",
                     "jde",
                     "kurzu",
                     "kurzů",
                     "ma",
                     "má",
                     "mam",
                     "mám",
                     "mohla",
                     "ni",
                     "ní",
                     "podzimní",
                     "podzimni",
                     "právě",
                     "prave",
                     "práve",
                     "pravě",
                     "psát",
                     "psat",
                     "query",
                     "kterých",
                     "kterych",
                     "která",
                     "ktera",
                     "ktere",
                     "které",
                     "který",
                     "ktery",
                     "kurz",
                     "zdali",
                     "na",
                     "ráda",
                     "rada",
                     "ramci",
                     "rámci",
                     "semestrální",
                     "semestralni",
                     "slo",
                     "šlo",
                     "sebou",
                     "své",
                     "sve",
                     "svou",
                     "svoji",
                     "svoje",
                     "sql",
                     "sql1",
                     "sql2",
                     "skutečně",
                     "skutečné",
                     "skutecne",
                     "tabulkami",
                     "tabulky",
                     "také",
                     "take",
                     "této",
                     "teto",
                     "tim",
                     "tím",
                     "timto",
                     "tímto",
                     "tomuto",
                     "tomto",
                     "toho",
                     "tohoto",
                     "tato",
                     "takže",
                     "takze",
                     "třeba",
                     "treba",
                     "trošku",
                     "trosku",
                     "testování",
                     "testing",
                     "úplně",
                     "uplne",
                     "určitě",
                     "urcite",
                     "udelat",
                     "udělat",
                     "úplně",
                     "uplne",
                     "uvest",
                     "uvést",
                     "vzorce",
                     "všude",
                     "vsude",
                     "vás",
                     "vas",
                     "vba",
                     "věci",
                     "veci",
                     "velmi",
                     "vim",
                     "vím",
                     "vic",
                     "víc", "více", "vice", "vizualizaci", "vizualizace", "vlastně", "vlastne", "vsechno", "všechno", "vůbec", "vubec", "výše", "vyse", "za", "zatim", "zatím", "zase", "zadne", "žádné", "zadna", "žádná", "zároveň", "zaroven", "znovu"]

# Initialize the TF-IDF Vectorizer with the custom stop words
tfidf_vectorizer_czech = TfidfVectorizer(stop_words=custom_stop_words)

# Fit and transform the 'expectation' column
tfidf_matrix_czech = tfidf_vectorizer_czech.fit_transform(df_motiv_czech_filtered['expectation'])

# Get the feature names (words) from the vectorizer
feature_names_czech = tfidf_vectorizer_czech.get_feature_names_out()

# Create a DataFrame to store TF-IDF values along with corresponding words
df_tfidf_czech = pd.DataFrame(tfidf_matrix_czech.toarray(), columns=feature_names_czech)

# Function to get the top N keywords based on TF-IDF values
def get_top_n_keywords(row, top_n=3):
    sorted_indices = row.argsort()[::-1][:top_n]  # Sort in descending order and get the top N indices
    top_keywords = [feature_names_czech[i] for i in sorted_indices]
    return ', '.join(top_keywords)

# Apply the function to the TF-IDF DataFrame to get top 3 keywords for each sentence
top_3_keywords_czech = df_tfidf_czech.apply(get_top_n_keywords, axis=1, top_n=3)

# Apply the function to the TF-IDF DataFrame to get top 2 keywords for each sentence
top_2_keywords_czech = df_tfidf_czech.apply(get_top_n_keywords, axis=1, top_n=2)

# Extract the top keyword (highest TF-IDF value) for each sentence
top_1_keyword_czech = df_tfidf_czech.idxmax(axis=1)

# Map the extracted keywords to the original DataFrame
df_motiv_czech_filtered['top_1_keyword'] = top_1_keyword_czech.values

In [46]:
# Remove diacritics
df_motiv_czech_filtered['top_1_keyword_no_diacritics'] = df_motiv_czech_filtered['top_1_keyword'].apply(unidecode)

In [47]:
for_translation = df_motiv_czech_filtered.groupby("top_1_keyword_no_diacritics").size().reset_index().sort_values(0, ascending=False)

In [48]:
for_translation.to_csv("for_translation_no_diacritics.csv", index=False)

In [49]:
relevant_participants_with_motivation = relevant_participants.merge(df_motiv_czech_filtered, on="participant_id", how="left")

In [50]:
relevant_participants_with_motivation.to_csv("participants.csv", index=False)

In [51]:
# df_motiv_czech_filtered = df_motiv_czech_filtered[["course_id", "participant_id", "age_group", "motivation", "participant_state_english", "got_IT_job_after_digital_academy", "top_1_keyword"]]

In [52]:
# for_translation = df_motiv_czech_filtered[["top_1_keyword"]].drop_duplicates().reset_index()

In [53]:
# for_translation.to_csv("for_translation.csv", index=False)
# # Save DataFrame to a .txt file
# for_translation.to_csv('words.txt', index=False, header=False, sep='\t', encoding='utf-8')

In [54]:
# czech_words_list = for_translation['top_1_keyword'].tolist()

In [55]:
# # Open a file in write mode ('w'). This will create a new file if it doesn't exist.
# with open("for_trans_list.txt", "w", encoding='utf-8') as f:
#     for word in czech_words_list:
#         f.write(f"{word}\n")  # Writing each word followed by a newline

In [56]:
# trans_1 = pd.read_csv('datasets/preklad/preklad_1_poe.txt', sep='-', header=None, names=['top_1_keyword', 'English'], on_bad_lines='skip')

In [57]:
# trans_1['top_1_keyword'] = trans_1['top_1_keyword'].str.replace(' ', '')

In [58]:
# preklad = pd.merge(for_translation, trans_1, on="top_1_keyword", how="left")

### Apify

In [59]:
# Initialize an empty list to hold the individual DataFrames
dfs = []

# Specify the folder containing the CSV files
folder_path = 'datasets/apify/apify_indeed-20231024T033726Z-001/apify_indeed/'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a CSV file
    if filename.endswith('.csv'):
        # Construct the complete filepath
        filepath = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(filepath)
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all the individual DataFrames into one DataFrame
final_df = pd.concat(dfs, ignore_index=True)

In [60]:
final_df_unique = final_df.drop_duplicates(subset=['id'], keep='last')

In [61]:
# VSECHNO LOWERCASE#
data = final_df_unique.applymap(lambda s: s.lower() if type(s) == str else s)

In [62]:
# List of skills to look for
skills_list = ['sql', 'spark', 'python', 'power bi', 'tableau', "qlik", "looker", "excel"]

# Function to check if a skill is mentioned in the description
def check_skills(description, skill):
    return skill.lower() in description.lower()

# Create new columns for each skill
for skill in skills_list:
    data[skill] = data['description'].apply(lambda x: check_skills(x, skill))


In [63]:
data["r"] = data['description'].str.contains(
    r'(?:\sr,|\sr-ko |\srkem |\sr-kem |\br+\s|\(r\b|\br\)|\br\/)', 
    regex=True, 
    flags=re.IGNORECASE
)
data["common_sense"] = data['description'].str.contains((r'selský rozum|common sense|(analytical(\s+|\w+){0,4}\s+(skills|abilities))|logical thinking|logické myšlení|analytické myšlení|(analytical (mindset|thinking))'), regex=True, flags=re.IGNORECASE)
data["scala"] = data['description'].str.contains(
    r'\bScala\b', 
    regex=True, 
    flags=re.IGNORECASE
)

C:\Users\fukter\AppData\Local\Temp\ipykernel_6800\1754805036.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data["common_sense"] = data['description'].str.contains((r'selský rozum|common sense|(analytical(\s+|\w+){0,4}\s+(skills|abilities))|logical thinking|logické myšlení|analytické myšlení|(analytical (mindset|thinking))'), regex=True, flags=re.IGNORECASE)


In [64]:
# # skill detection
# data["SQL"]= data['description'].str.contains((r'sql+[^a-z]|sql+[^a-z]|SQLkem'), regex=True, flags=re.IGNORECASE)
# data["Python"]= data['description'].str.contains((r'Python'), regex=True, flags=re.IGNORECASE)
# data["Julia"]= data['description'].str.contains((r'Julia'), regex=True, flags=re.IGNORECASE)
# data["Spark"]= data['description'].str.contains((r' spark+[^a-z]|[^a-z]spark'), regex=True, flags=re.IGNORECASE)
# data["Scala"]= data['description'].str.contains((r'Scala+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["Alteryx"]= data['description'].str.contains((r'alteryx+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["Tableau"]= data['description'].str.contains((r'tableau+[^a-z]|tablo+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["PowerBI"]= data['description'].str.contains((r'Power BI+[^a-z]|PowerBI+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["Qlik"]= data['description'].str.contains((r'Qlik+[^a-z]|Qlik+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["R"]= data['description'].str.contains((r'R-ko | Rkem | R-kem |\br+\s|\(r\b|\br\)|\br\/'), regex=True, flags=re.IGNORECASE)
# data["Oracle"]= data['description'].str.contains((r'oracle+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["Snowflake"]= data['description'].str.contains((r'Snowflake+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["MySQL"]= data['description'].str.contains((r' MySQL+[^a-z]|My SQL+[^a-z]|mssql|ms-sql'), regex=True, flags=re.IGNORECASE)
# data["Postgres"]= data['description'].str.contains((r'postgre+[^a-z]|postgres+[^a-z]|postgresql+[^a-z]|possgrest'), regex=True, flags=re.IGNORECASE)
# data["Excel"]= data['description'].str.contains((r'excel+[^a-z]|excelov[ý,ým]|xls'), regex=True, flags=re.IGNORECASE) # excellence to nebere uz tak jak to je, bere to i Excel, Excel; atd
# data["UML"]= data['description'].str.contains((r'UML+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["Enterprise Architect"]= data['description'].str.contains((r'\benterprise architect\b'), regex=True, flags=re.IGNORECASE)
# data["Cognos"]= data['description'].str.contains((r'cognos+[^a-z]'), regex=True, flags=re.IGNORECASE)
# data["GIT"]= data['description'].str.contains((r'\bGIT\b|\bgitlab\b|\bgithub\b'), regex=True, flags=re.IGNORECASE)
# data["AWS"]= data['description'].str.contains((r'\bAWS\b|\bAmazon Web Services\b|\bAmazon Web\b'), regex=True, flags=re.IGNORECASE)
# data["Azure"]= data['description'].str.contains((r'azure'), regex=True, flags=re.IGNORECASE)
# data["CommonSense"] = data['description'].str.contains((r'selský rozum|common sense|(analytical(\s+|\w+){0,4}\s+(skills|abilities))|logical thinking|logické myšlení|analytické myšlení|(analytical (mindset|thinking))'), regex=True, flags=re.IGNORECASE)
# data["DataAnalysis"] = data['description'].str.contains((r'analýza dat|datová analýza|(analyzovat(\s|\w+){0,2}\s+data)|(analyzováním(\s+|\w+){0,2}\s+dat)|data analysis|datových analytiků|datové analýzy|datová analytika|analytika dat|datovou analytiku|datové analytiky|data analyst'), regex=True, flags=re.IGNORECASE)

In [65]:
# Define a function for custom job categorization based on the conditions provided
def custom_job_categorization(job_name_lower):
    
    if (("it" in job_name_lower and "spec" in job_name_lower) or
          "it analy" in job_name_lower or
          ("it" in job_name_lower and "support" in job_name_lower) or
          ("it" in job_name_lower and "admin" in job_name_lower) or
          ("it" in job_name_lower and "správc" in job_name_lower) or
          ("it" in job_name_lower and "techni" in job_name_lower) or
          "devops" in job_name_lower or
          ("it" in job_name_lower and "syst" in job_name_lower) or
          "it " in job_name_lower):
        return "Not Relevant Positions"
    
    elif ("anal" in job_name_lower or
      "data " in job_name_lower or
      "datový " in job_name_lower or
      "datová " in job_name_lower or
      (("bi" in job_name_lower or "dat" in job_name_lower or "dát" in job_name_lower) and
       ("power" in job_name_lower or
        "onsult" in job_name_lower or
        "anal" in job_name_lower or
        "onzult" in job_name_lower or
        "develop" in job_name_lower or
        "data" in job_name_lower or
        "exp" in job_name_lower or
        "design" in job_name_lower or
        "program" in job_name_lower or
        "manag" in job_name_lower or
        "admin" in job_name_lower or
        "archit" in job_name_lower or
        "engineer" in job_name_lower or
        "inž" in job_name_lower or
        "mod" in job_name_lower or
        "special" in job_name_lower or
        "lead" in job_name_lower or
        "etl" in job_name_lower))):
        return "Data Analytics / Business Intelligence / Reporting"

    elif (("business" in job_name_lower and ("anal" in job_name_lower or "intell" in job_name_lower)) or
      "reporting" in job_name_lower or
      "sql" in job_name_lower or
      "qlik" in job_name_lower or
      "big data" in job_name_lower):
        return "Data Analytics / Business Intelligence / Reporting"
        
    elif "test" in job_name_lower or "quality" in job_name_lower or "qa" in job_name_lower:
        return "Testing"
    
    elif (("javascript" in job_name_lower) or
          ("php" in job_name_lower) or
          ("front" in job_name_lower and "end" in job_name_lower) or
          (".js" in job_name_lower) or
          ("wordpress" in job_name_lower) or
          ("angular" in job_name_lower) or
          ("react" in job_name_lower) or
          ("web" in job_name_lower and ("develop" in job_name_lower or "program" in job_name_lower or "master" in job_name_lower or "vývoj" in job_name_lower or "kod" in job_name_lower))):
        return "Web & Frontend Development"
    
    elif (("machine" in job_name_lower and "learning" in job_name_lower)  or
          ("datov" in job_name_lower and "věd" in job_name_lower)  or
          "ai/ml" in job_name_lower or
          "ai " in job_name_lower or
          "data science" in job_name_lower or
          "artificial" in job_name_lower or
          ("stroj" in job_name_lower and "učen" in job_name_lower)):
        return "Data Science / Machine Learning / Artificial Intelligence"
    
    elif ("recruit" in job_name_lower or
      ("back" in job_name_lower and "office" in job_name_lower) or
      "konstrukt" in job_name_lower or
      "r&d" in job_name_lower or
      "compliance" in job_name_lower):
        return "Not Relevant Positions"
    
    elif "security" in job_name_lower or "kyber" in job_name_lower or "cyber" in job_name_lower or "bezpeč" in job_name_lower or "data privacy" in job_name_lower or "data protection" in job_name_lower:
        return "Cybersecurity"
    
    elif "android" in job_name_lower or "ios" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "salesforce" in job_name_lower:
        return "Marketing Technology"
    
    elif "sap" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "proje" in job_name_lower or "produ" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "scrum" in job_name_lower:
        return "Not Relevant Positions"
    
    elif "dwh" in job_name_lower or "datab" in job_name_lower or "db " in job_name_lower:
        return "Data Analytics / Business Intelligence / Reporting"
    
    elif ("graph" in job_name_lower or
          "graf" in job_name_lower or
          " ux" in job_name_lower or
          "ux " in job_name_lower or
          "design" in job_name_lower or
          " ui" in job_name_lower or
          "ui " in job_name_lower or
          "user experience" in job_name_lower):
        return "UX / UI / Graphic Design"
    
    elif ("full" in job_name_lower and "stack" in job_name_lower) or "fullstack" in job_name_lower:
        return "Backend Development"
    
    elif ("c#" in job_name_lower or
      ".net" in job_name_lower or
      "java" in job_name_lower or
      "javist" in job_name_lower or
      "javě" in job_name_lower or
      "python" in job_name_lower or
      "c++" in job_name_lower or
      ("back" in job_name_lower and "end" in job_name_lower) or
      "software" in job_name_lower):
        return "Backend Development"
    
    elif "marke" in job_name_lower or "social" in job_name_lower or "ppc" in job_name_lower or "seo" in job_name_lower or "rtb" in job_name_lower:
        return "Marketing Technology"
    
    else:
        return "Not Relevant Positions"

In [66]:
data["position_category"] = data["positionName"].apply(custom_job_categorization)

In [67]:
data = data[data["position_category"]=="Data Analytics / Business Intelligence / Reporting"]

In [68]:
# data[["positionName", "position_category"]].drop_duplicates().to_csv("job_names.csv")

In [69]:
data = data[["id", "description", "sql", "spark", "python", "power bi", "tableau", "scala", "qlik", "looker", "excel", "r", "common_sense"]]

In [70]:
len(data)

519

In [71]:
# Columns you want to consider for counting True values
cols_to_count = ["sql", "spark", "python", "power bi", "tableau", "scala", "qlik", "looker", "excel", "r", "common_sense"]

# Count the number of True values per row in the selected columns only
data['True_count'] = data[cols_to_count].sum(axis=1)

In [72]:
data.head(3)

,id,description,sql,spark,python,power bi,tableau,scala,qlik,looker,excel,r,common_sense,True_count
647,0fc34e9a7cbb6294,pro oddělení business development hledáme nadš...,False,False,False,False,False,False,False,False,True,False,False,1
882,38150ddcde93376b,"jsme česká společnost, zabývající se zejména v...",True,False,False,True,False,False,False,False,False,False,True,3
903,fd0ded863ed353f0,"ahoj, tady michal z týmu zoner photo studia. t...",False,False,False,False,False,False,False,False,False,False,False,0


In [73]:
data = data[data["True_count"]>0]

In [74]:
data = data[["id", "sql", "spark", "python", "power bi", "tableau", "scala", "qlik", "looker", "excel", "r", "common_sense"]]

In [75]:
len(data)

447

In [76]:
# # Calculate the total number of rows in the DataFrame
# total_rows = len(data)
# # Add a new column to show the share of rows where each skill is True
# skill_counts_df['Share'] = skill_counts_df['Count'] / total_rows * 100
# # Count the number of True values for each skill (ignoring the 'id' column)
# skill_counts = data.drop(columns='id').sum()
# # Convert the Series to a DataFrame
# skill_counts_df = skill_counts.reset_index()
# skill_counts_df.columns = ['Skill', 'Count']

In [77]:
data.to_csv("analytics_skills_job_ads.csv", index=False)